In [2]:
import json
import numpy as np
import random

In [3]:
def read_json(data_file):
    try:
        with open(data_file, 'r' ,encoding='utf-8') as file:
            content = json.load(file)
        return content
    except Exception as e:
        # 输出错误信息
        print("错位信息为:{}".format(e))
        # 尝试逐行读取
        try:
            content = []
            with open(data_file, 'r' ,encoding='utf-8') as file:
                for line in file:
                    content.append(json.loads(line))
            return content
        except Exception as e:
             print("错位信息为:{}".format(e))

In [3]:
# 获取初始数据
datas_to_trans = read_json("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content.json")
datas_to_trans2 = read_json("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content2.json")
datas_to_trans3 = read_json("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content3.json")
datas_to_trans4 = read_json("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Pass_content4.json")

In [4]:
datas_to_trans = datas_to_trans + datas_to_trans2 + datas_to_trans3 + datas_to_trans4
len(datas_to_trans)

15884

In [5]:
head_template = [
    "请你根据给出的学术全文本内容，生成关于该文本的摘要。",
    "请根据提供的学术文本内容，生成一个该文本的摘要。",
    "请根据给定的学术全文，撰写一个简要总结。",
    "请根据所提供的学术文章内容，生成一个概述。",
    "你需要阅读以下学术文本，并根据其内容编写摘要",
    "根据给出的学术材料，请你整理出一份摘要。",
    "请将下面的学术文本内容总结成一个简短的摘要。",
    "请从提供的学术文本中提炼出摘要内容。",
    "请根据以下学术文本写一段简短的总结。",
    "请基于下列学术全文，提炼并生成相关摘要。"
]

In [6]:
train_data = []
for value in datas_to_trans:
    if len(value['content'].split('\n\n\n\n\n')[1]) < 10000:
        continue
    random_number = random.randint(0, 9)
    QA_dict = {}
    QA_data = value['content']
    conversations = []
    # 构造单次问答对
    first_QA = {}
    first_QA['from'] = "human"
    first_QA["value"] = "{}\n-你将获得的文本内容为：{}".format(head_template[random_number],value['content'].split('\n\n\n\n\n')[1])
    second_QA = {}
    second_QA['from'] = "gpt"
    second_QA["value"] = value['abstract']
    conversations.append(first_QA)
    conversations.append(second_QA)
    QA_dict['conversations'] = conversations
    QA_dict['system'] = ""
    train_data.append(QA_dict)

In [7]:
with open('/gemini/code/Abstract_extract/ProcessedData/rdfybk/train_data_abstract_mulIns_big.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=2)

In [12]:
np.mean([len(value['conversations'][0]['value']) for value in train_data])

15048.124137931034

## 新增实验

In [4]:
original_data = read_json("/gemini/code/Abstract_extract/ProcessedData/rdfybk/train_data_abstract_mulIns_small.json")
Model72B_output = read_json("/gemini/code/Abstract_extract/ProcessedData/rdfybk/Model_outputs_72B_cot_plus.json")

In [5]:
new_train_data_abstract_mulIns_small = []
for value1, value2 in zip(original_data[:1000], Model72B_output):
    cot_str = value2.split('<最终摘要>')[0]
    ori_zuizhong = value1['conversations'][1]['value']
    value1['conversations'][1]['value'] = cot_str + '<最终摘要>' + ori_zuizhong
    new_train_data_abstract_mulIns_small.append(value1)

In [10]:
with open('/gemini/code/Abstract_extract/ProcessedData/rdfybk/new_train_data_abstract_mulIns_small.json', 'w', encoding='utf-8') as f:
    json.dump(new_train_data_abstract_mulIns_small, f, ensure_ascii=False, indent=2)